<a href="https://colab.research.google.com/github/BabiiIn/NLP/blob/main/%D0%A7%D0%B0%D1%81%D1%82%D1%8C_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Реализация классической ML-модели поверх векторов из  BERT-модели**

Задача 3.
1. Обучить модель классического ML на основе векторов, полученной из предобученной BERT- модели.  
2. Дообучить предобученную модель на данных из задачи.
3. Посчитать метрики.


Работа выполнена с использованием статей:  
https://towardsdatascience.com/distilling-bert-how-to-achieve-bert-performance-using-logistic-regression-69a7fc14249d  

https://towardsdatascience.com/bert-to-the-rescue-17671379687f

Основная идея состоит в том, чтобы дообучить на наших данных BERT-модель, затем использовать необработанные прогнозы (т.е. предсказания перед конечной функцией активации) дообученной BERT-модели для обучения классической ML-модели.  
Таким образом, чтобы решить поставленную задачу и добиться необходимого accuracy, мы попытаемся добиться высокой производительности BERT-модели на классической ML-модели.

# Импорт библиотек


In [ ]:
!pip install pytorch_pretrained_bert pytorch-nlp

In [ ]:
%matplotlib inline
import sys
import itertools
import numpy as np
import pandas as pd
import random as rn
import matplotlib.pyplot as plt
import torch
import tensorflow as tf

from sklearn.utils.extmath import softmax
from pytorch_pretrained_bert import BertModel
from torch import nn
from torchnlp.datasets import imdb_dataset
from pytorch_pretrained_bert import BertTokenizer
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Загружаем очищенные данные

Данные были подготовлены в рамках выполнения Задачи 2

In [ ]:
train = pd.read_csv('/content/df.csv').dropna()
train.drop(train.columns[0], axis=1, inplace=True)
train

,text,label
0,and and,Neutral
1,advice talk to your neighbours family to excha...,Positive
2,coronavirus australia woolworths to give elder...,Positive
3,my food stock is not the only one which is emp...,Positive
4,me ready to go at supermarket during the covid...,Extremely Negative
...,...,...
40863,airline pilots offering to stock supermarket s...,Neutral
40864,response to complaint not provided citing covi...,Extremely Negative
40865,you know it s getting tough when is rationing ...,Positive
40866,is it wrong that the smell of hand sanitizer i...,Neutral


In [ ]:
# Количество категорий для классификации
num_classes = len(train.label.value_counts())
print('Количество категорий для классификации: {}'.format(num_classes))

Количество категорий для классификации: 5


# Предобработка данных

Разобьем исходные данные на 2 части: 20% - part_1 и 80% -part_2.  

Данные в размере 20% от выборки (part_1) будем использовать для дообучения BERT-модели на наших данных.  

Данные в размере 80% от выборки (part_2) будем использовать для получения из дообученной BERT-модели необработанных logits, которые далее применим для обучения классической ML-модели.


In [ ]:
# Разбиваем данные на 2 части в пропорции 20/80

part_1, part_2 = train_test_split(train, train_size=.2)
part_1.shape, part_2.shape

((8173, 2), (32695, 2))

In [ ]:
# Кодируем категориальные признаки - метки классов
df_ohe_y = part_1['label'].copy()
y_ohe = pd.get_dummies(df_ohe_y)
print('Размерность y_ohe:', y_ohe.shape)

Размерность y_ohe: (8173, 5)


In [ ]:
# Из первой части данных (part_1) формируем тренировочные и проверочные данные

X_train, X_test, y_train, y_test = train_test_split(
    part_1["text"].values.tolist(),
    y_ohe.values.tolist(),
    test_size=.5
)
len(X_train), len(X_test), len(y_train), len(y_test)

(4086, 4087, 4086, 4087)

In [ ]:
# Создаем токенизатор

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 24268092.83B/s]


Токенизируем каждый набор данных, добавляем специальный токен, а затем принимает только первые 512 токенов как для обучающих, так и для тестовых наборов

In [ ]:
train_tokens = list(map(lambda t: ['[CLS]']
                   + tokenizer.tokenize(t)[:511], X_train))
test_tokens = list(map(lambda t: ['[CLS]']
                   + tokenizer.tokenize(t)[:511], X_test))

len(train_tokens), len(test_tokens)

(4086, 4087)

Преобразуем каждый токен в id, существующий в словаре токенизатора

In [ ]:
train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))

Дополняем данные до единого размера  - 512 токенов

In [ ]:
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512,
                                truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post",
                               padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((4086, 512), (4087, 512))

In [ ]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

In [ ]:
# Преобразуем метки классов в numpy-массивы
train_y = np.array(y_train)
test_y = np.array(y_test)
train_y.shape, test_y.shape

((4086, 5), (4087, 5))


# Cоздаем Baseline с помощью логистической регрессии

In [ ]:
# Переводим метки классов в одномерный тензор.
y_train_1 = tf.argmax(train_y, axis = 1)
print(len(y_train_1))
y_test_1 = tf.argmax(test_y, axis = 1)
print(len(y_test_1))

4086
4087


In [ ]:
# Создаем и обучаем модель с помощью Pipeline
baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)),
                              LogisticRegression()).fit(X_train, y_train_1)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Делаем предсказание на исходных данных задачи
baseline_predicted = baseline_model.predict(X_test)

In [ ]:
# Выводим classification_report
print(classification_report(y_test_1, baseline_predicted))

              precision    recall  f1-score   support

           0       0.54      0.38      0.44       558
           1       0.54      0.30      0.39       645
           2       0.40      0.36      0.38      1024
           3       0.47      0.60      0.52       768
           4       0.37      0.48      0.42      1092

    accuracy                           0.43      4087
   macro avg       0.46      0.42      0.43      4087
weighted avg       0.45      0.43      0.43      4087



Получаем результаты: Классическая модель ML предсказывает классы с accuracy 0.43 и f1-score 0.43.  
Это базовый вариант, здесь мы не используем вектора из предобученной BERT-модели.


# Тонкая настройка BERT model

Осуществим «тонкую настройку» BERT - добавим дополнительный слой поверх BERT, а затем обучим все это вместе. Таким образом, мы тренируем наш дополнительный слой, а также изменяем (тонко настраиваем) веса BERT.

Результатом тонкой настройки BERT является обученная модель, которая использует BERT и дополнительный линейный слой для обеспечения классификации на 5 классов

In [ ]:
# Создаем модель BERT
class BertMultiLabelClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertMultiLabelClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.linear = nn.Linear(768, num_classes)

    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks,
                                    output_all_encoded_layers=False)
        linear_output = self.linear(pooled_output)
        return linear_output

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# Создаем модель и перемещаем ее в GPU

bert_clf = BertMultiLabelClassifier()
bert_clf = bert_clf.cuda()

100%|██████████| 407873900/407873900 [00:05<00:00, 75991771.70B/s]


In [ ]:
# Создаем тензоры для передачи в модель

train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'439.077376M'

In [ ]:
BATCH_SIZE = 4
EPOCHS = 3

In [ ]:
# Готовим загрузчики данных
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor,
                             train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler,
                             batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor,
                            test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler,
                            batch_size=BATCH_SIZE)


In [ ]:
# Берем оптимизатор Adam
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [ ]:
# Применяем функцию потерь
loss_func = nn.BCEWithLogitsLoss().cuda()

In [ ]:
# Обучаем модель на 3 эпохах

losses = []
steps = []
step = 0
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        probas = bert_clf(token_ids, masks)    # получаем вероятности

        batch_loss = loss_func(probas, labels) # рассчитаем потери
        train_loss += batch_loss.item()


        bert_clf.zero_grad()      # обнуляем градиенты с предыдущего шага
        batch_loss.backward()     # рассчитаем и распространим новые градиенты


        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()     # обновим параметры модели относительно градиентов

        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("{0}/{1} loss: {2} ".format(step_num, len(X_train) / BATCH_SIZE,
                                         train_loss / (step_num + 1)))
        losses.append(batch_loss.item())
        steps.append(step)
        step += 1

Epoch:  3
1021/1021.5 loss: 0.16568873050110505 


In [ ]:
# Тестируем результаты работы модели

bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        probas = bert_clf(token_ids, masks)
        numpy_probas = probas.cpu().detach().numpy()
        bert_predicted += list(np.argmax(softmax(numpy_probas), axis=1))

In [ ]:
# Проверим как обучилась модель
print(classification_report(y_test_1, bert_predicted))

              precision    recall  f1-score   support

           0       0.55      0.79      0.65       558
           1       0.62      0.70      0.66       645
           2       0.55      0.48      0.51      1024
           3       0.66      0.66      0.66       768
           4       0.56      0.46      0.51      1092

    accuracy                           0.59      4087
   macro avg       0.59      0.62      0.60      4087
weighted avg       0.59      0.59      0.58      4087



В результате точной настройки BERT на нашем маркированном наборе получили результативность: accuracy 0.59, f1-score - 0.60. Результаты оказались лучше, чем базовый уровень (Baseline)

# Получим необработанные logits из дообученной BERT-модели на тренировочных данных

In [ ]:
# Готовим загрузчики тренировочных данных
train_dataset_for_distill = TensorDataset(train_tokens_tensor,
                                         train_masks_tensor, train_y_tensor)
train_dataloader_for_distill = DataLoader(train_dataset, batch_size=BATCH_SIZE)

In [ ]:
bert_clf.eval()
train_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(train_dataloader_for_distill):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        numpy_logits = logits.cpu().detach().numpy()

        train_logits.append(numpy_logits)
train_logits = np.vstack(train_logits)

In [ ]:
train_logits.shape

(4086, 5)

# Реализуем классическую ML-модель на необработанных logits из "тонко настроенной" BERT-модели

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
distilled_model = make_pipeline(CountVectorizer(ngram_range=(1,3)),
                               LinearRegression()).fit(X_train, train_logits)

In [ ]:
distilled_predicted_logits = distilled_model.predict(X_test)

In [ ]:
# Выведем classification_report
print(classification_report(y_test_1,
                           np.argmax(softmax(distilled_predicted_logits),
                           axis=1)))

              precision    recall  f1-score   support

           0       0.48      0.54      0.51       558
           1       0.51      0.56      0.53       645
           2       0.39      0.28      0.32      1024
           3       0.40      0.72      0.52       768
           4       0.41      0.24      0.31      1092

    accuracy                           0.43      4087
   macro avg       0.44      0.47      0.44      4087
weighted avg       0.43      0.43      0.41      4087



Результативность работы модели невысокая: accuracy 0.43, f1-score - 0.44. Это не лучше Baseline.

#Обучение классической ML-модели с использованием необработанных logits

В этой части работы мы используем вторую часть датасета (80%) в качестве непомеченного набора и «маркируем» его с помощью нашей тонко настроенной модели BERT

Здесь мы будем использовать конечный выход BERT в качестве входных данных для  ML-модели.

In [ ]:
unlabeled_data = part_2.text
unlabeled_data.shape

(32695,)

In [ ]:
unlabeled_tokens = list(map(lambda t: ['[CLS]']
                           + tokenizer.tokenize(t)[:511], unlabeled_data))

In [ ]:
unlabeled_tokens_ids = list(map(tokenizer.convert_tokens_to_ids,
                               unlabeled_tokens))

In [ ]:
unlabeled_tokens_ids = pad_sequences(unlabeled_tokens_ids, maxlen=512,
                                    truncating="post", padding="post",
                                    dtype="int")
unlabeled_tokens_ids.shape


(32695, 512)

In [ ]:
unlabeled_masks = [[float(i > 0) for i in ii] for ii in unlabeled_tokens_ids]

In [ ]:
unlabeled_tokens_tensor = torch.tensor(unlabeled_tokens_ids)
unlabeled_masks_tensor = torch.tensor(unlabeled_masks)

In [ ]:
unlabeled_dataset = TensorDataset(unlabeled_tokens_tensor,
                                 unlabeled_masks_tensor)
unlabeled_dataloader = DataLoader(unlabeled_dataset, batch_size=BATCH_SIZE)

Из немаркированного набора (unlabeled_data) с помощью настроенной модели BERT получаем необработанные logits

In [ ]:
bert_clf.eval()
unlabeled_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(unlabeled_dataloader):

        token_ids, masks = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        numpy_logits = logits.cpu().detach().numpy()

        unlabeled_logits.append(numpy_logits)
        clear_output(wait=True)
        print("{0}/{1}".format(step_num, len(unlabeled_data) / BATCH_SIZE))
unlabeled_logits = np.vstack(unlabeled_logits)

8173/8173.75


In [ ]:
unlabeled_logits.shape

(32695, 5)

Обучаем модель LinearRegression на полученных необработанных logits

In [ ]:
unlabeled_model = make_pipeline(CountVectorizer(ngram_range=(1,3)),
                               LinearRegression()).fit(unlabeled_data,
                               unlabeled_logits)

Делаем предсказание на данных X_test из части датасета part_1

In [ ]:
unlabele_predicted_logits = unlabeled_model.predict(X_test)

In [ ]:
unlabele_predicted_logits.shape

(4087, 5)

In [ ]:
print(classification_report(y_test_1,
                           np.argmax(softmax(unlabele_predicted_logits),
                           axis=1)))

              precision    recall  f1-score   support

           0       0.52      0.63      0.57       558
           1       0.53      0.61      0.57       645
           2       0.44      0.30      0.36      1024
           3       0.44      0.72      0.54       768
           4       0.44      0.28      0.35      1092

    accuracy                           0.47      4087
   macro avg       0.47      0.51      0.48      4087
weighted avg       0.47      0.47      0.45      4087



Результативность работы модели составила: accuracy 0.47, f1-score - 0.48. Это несколько выше ранее достигнутого результата.

К сожалению, требуемую метрику результативности работы модели  accuracy >= 85% достичь не удалось.

#Итоговые метрики всех моделей, реализованных в проекте (Части 1, 2 и 3)

In [ ]:
# Выведем таблицу результатов работы всех моделей.

table=pd.DataFrame(columns = ['Accuracy', 'f1-score'],
                   index = ['Наивная модель', 'MultinomialNB', 'SGDClassifier',
                            'Модель 1 LSTM', 'Модель 2 LSTM',
                            'Модель 3 с двумя LSTM', 'Модель 4 с двумя LSTM',
                            'Модель 5 с RNN,LSTM', 'Модель 6 с GRU',
                            'Модель 7 Bidirectional LSTM', 'SimpleRNN', 'LSTM',
                            'GRU', 'LinearRegression на векторах BERT', 'Дообученная LinearRegression на векторах BERT'
                           ]
                  )

In [ ]:
table.loc['Наивная модель'] = [0.20, 0.09]
table.loc['MultinomialNB'] = [0.37, 0.22]
table.loc['SGDClassifier'] = [0.54, 0.54]
table.loc['Модель 1 LSTM'] = [0.65, 0.66]
table.loc['Модель 2 LSTM'] = [0.62, 0.63]
table.loc['Модель 3 с двумя LSTM'] = [0.61, 0.62]
table.loc['Модель 4 с двумя LSTM'] = [0.63, 0.64]
table.loc['Модель 5 с RNN,LSTM'] = [0.63, 0.64]
table.loc['Модель 6 с GRU'] = [0.28, 0.09]
table.loc['Модель 7 Bidirectional LSTM'] = [0.62, 0.63]
table.loc['SimpleRNN'] = [0.25, 0.12]
table.loc['LSTM'] = [0.72, 0.72]
table.loc['GRU'] = [0.70, 0.71]
table.loc['LinearRegression на векторах BERT'] = [0.43, 0.44]
table.loc['Дообученная LinearRegression на векторах BERT'] = [0.47, 0.48]

In [ ]:
table

,Accuracy,f1-score
Наивная модель,0.2,0.09
MultinomialNB,0.37,0.22
SGDClassifier,0.54,0.54
Модель 1 LSTM,0.65,0.66
Модель 2 LSTM,0.62,0.63
Модель 3 с двумя LSTM,0.61,0.62
Модель 4 с двумя LSTM,0.63,0.64
"Модель 5 с RNN,LSTM",0.63,0.64
Модель 6 с GRU,0.28,0.09
Модель 7 Bidirectional LSTM,0.62,0.63


Из всех рассмотренных в Задаче (Части 1, 2 и 3) методов наилучшие результаты достигнуты при использовании  предварительно обученных векторных представлений слов GloVe моделью LSTM (f1-score 0.72, accuracy 0.72).  
К сожалению, моделью классического ML на основе веторов из предобученной BERT- модели (в том числе с дообучением) улучшить данный результат не удалось.